# Workflow

In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import sys

sys.path.append('../hydra_tod/')

from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u
from utils import Leg_poly_proj, view_samples
from flicker_model import sim_noise, flicker_cov
from simulation import * 

In [2]:
# Import the FlickerCorrEmulator class first
from flicker_model import FlickerCorrEmulator

# Then import from noise_sampler_fixed_fc
from noise_sampler_fixed_fc import flicker_noise_sampler, flicker_likeli_func

Using the emulator for flicker noise correlation function.


In [3]:
# Load the simulation results
import pickle
from simulation import TODSimulation

with open('tod_simulation_single.pkl', 'rb') as f:
    tod_sim = pickle.load(f)


In [4]:
sky_params = tod_sim.sky_params
nd_rec_params = tod_sim.nd_rec_params
gain_params_setting = tod_sim.gain_params_setting
calibration_indices = tod_sim.calibration_1_indice
alpha = tod_sim.alpha

TOD_setting = tod_sim.TOD_setting
t_list = tod_sim.t_list
gain_proj = tod_sim.gain_proj
Tsky_operator_setting = tod_sim.Tsky_operator_setting
nd_rec_operator = tod_sim.nd_rec_operator

logfc = tod_sim.logfc

# Additional variables needed for the analysis
pixel_indices = tod_sim.pixel_indices
nside = tod_sim.nside
f0 = tod_sim.f0
T_ndiode = tod_sim.T_ndiode
rec_params = tod_sim.rec_params
logf0_list = [tod_sim.logf0]  # Convert to list format as expected later

In [5]:
init_Tsys_params = np.hstack([sky_params, nd_rec_params])
init_noise_params = [-4.8, 2.2]

gain_prior_cov_inv = np.zeros(4)
# prior_std_frac = 1
# gain_prior_cov_inv = 1. / (prior_std_frac * gain_params_setting)**2  # Prior STD ~100%
gain_prior_cov_inv[0] = 2.51 # DC mode: Prior STD ~10%
gain_prior_mean = gain_params_setting

prior_std_frac = 0.1
Tsky_prior_cov_inv = 1. / (prior_std_frac * sky_params)**2
Tsky_prior_cov_inv[calibration_indices] = 1e20

aux = np.ones_like(nd_rec_params)*0.0
aux[0] = 1
Trec_prior_cov_inv = [aux] # Effectlively no prior


In [6]:
# Import the FlickerCorrEmulator class first
from flicker_model import FlickerCorrEmulator
from full_Gibbs_sampler import full_Gibbs_sampler_multi_TODS_v2

Tsys_samples, all_gain_samples, all_noise_samples = \
    full_Gibbs_sampler_multi_TODS_v2([TOD_setting],
                                    [t_list],
                                    [gain_proj],
                                    [Tsky_operator_setting],
                                    [nd_rec_operator],
                                    init_Tsys_params,
                                    init_noise_params, 
                                    [logfc],
                                    num_Jeffrey=False,
                                    wnoise_var=2.5e-6,
                                    Tsky_prior_cov_inv=Tsky_prior_cov_inv,
                                    Tsky_prior_mean=sky_params,
                                    local_Trec_prior_cov_inv_list=Trec_prior_cov_inv,
                                    local_Trec_prior_mean_list=[nd_rec_params],
                                    local_gain_prior_cov_inv_list=[gain_prior_cov_inv],
                                    local_gain_prior_mean_list=[gain_prior_mean],
                                    #local_noise_prior_func_list=[log_prior_noise, log_prior_noise],
                                    n_samples=1000,
                                    tol=1e-12,
                                    Est_mode=False)

# First, save the samples as npy files..

np.save("outputs/GS1/Tsys_samples.npy", Tsys_samples)
np.save("outputs/GS1/gain_samples.npy", np.concatenate(all_gain_samples, axis=0))
np.save("outputs/GS1/noise_samples.npy", np.concatenate(all_noise_samples, axis=0))

Using the emulator for flicker noise correlation function.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 0: [6.31207126 0.42006735 0.26267015 0.05346635]
Rank: 0, local id: 0, noise_sample 0: [-5.18846867  1.79728059]
Reached max iterations with fractional norm error 0.00345530965937368.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 1: [6.31204512 0.41925703 0.26270152 0.05372414]
Rank: 0, local id: 0, noise_sample 1: [-4.05524185  3.39923549]
Reached max iterations with fractional norm error 7.364261813799272e-05.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 2: [6.3116968  0.41984427 0.26125152 0.05132002]
Rank: 0, local id: 0, noise_sample 2: [-5.4179771   2.05450374]
Reached max iterations with fractional norm error 0.0008093521929151466.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 3: [6.3114093  0.42031054 0.262032   0.05249473]
Rank: 0, local id: 0, noise_sample 3: [-5.04313672  2.60853435]
Reached max iterations with 

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]


Rank: 0, local id: 0, noise_sample 9: [-4.09272247  3.16443907]
Reached max iterations with fractional norm error 0.0012108003676154263.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 10: [6.3121813  0.41815133 0.2610602  0.05168703]
Rank: 0, local id: 0, noise_sample 10: [-4.5011056   2.42732746]
Reached max iterations with fractional norm error 0.0011730717221425171.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 11: [6.31180633 0.41891204 0.25869201 0.04750106]
Rank: 0, local id: 0, noise_sample 11: [-4.43113511  2.2809869 ]
Reached max iterations with fractional norm error 0.0011236814990006852.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 12: [6.31145944 0.41826241 0.25598916 0.04698488]
Rank: 0, local id: 0, noise_sample 12: [-5.15903608  1.71637157]
Reached max iterations with fractional norm error 0.0007300889240599501.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 13: [6.31110769 0.41805821 0.25603985 0.04293938]
Rank:

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)


Rank: 0, local id: 0, noise_sample 35: [-3.08784015  3.97697834]
Reached max iterations with fractional norm error 0.0028133357223378034.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 36: [6.31220277 0.43391303 0.31744779 0.02647477]
Rank: 0, local id: 0, noise_sample 36: [-3.2466569   3.19540899]
Reached max iterations with fractional norm error 0.00023663043089051198.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 37: [6.30751626e+00 4.22419053e-01 2.60523527e-01 7.11328383e-04]
Rank: 0, local id: 0, noise_sample 37: [-3.8674597   2.37350245]
Reached max iterations with fractional norm error 0.0017819661326259214.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 38: [6.30708626 0.41716894 0.26178458 0.01634455]
Rank: 0, local id: 0, noise_sample 38: [-4.27370371  1.96664427]
Reached max iterations with fractional norm error 0.0008964668197334738.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 39: [6.30905721 0.4206141  0.27498452

...

Rank: 0, local id: 0, noise_sample 963: [-4.09954154  2.9397429 ]
Reached max iterations with fractional norm error 0.00014197832531397749.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 964: [6.33973574 0.42163885 0.2714496  0.04552306]
Rank: 0, local id: 0, noise_sample 964: [-4.41975399  2.36139317]
Reached max iterations with fractional norm error 2.876323792398451e-05.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 965: [6.33915251 0.42132616 0.26562382 0.04695578]
Rank: 0, local id: 0, noise_sample 965: [-5.97691155  1.26760785]
Reached max iterations with fractional norm error 0.0002656596320655099.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 966: [6.33938206 0.4231043  0.26540192 0.04697705]
Rank: 0, local id: 0, noise_sample 966: [-5.07094765  1.7951135 ]
Reached max iterations with fractional norm error 0.0007703089477383373.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 967: [6.33946217 0.42301488 0.26379375 0.04689

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)


Rank: 0, local id: 0, noise_sample 986: [-4.57290273  3.09411127]
Reached max iterations with fractional norm error 0.0004671332177565198.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 987: [6.34038293 0.42368042 0.25942494 0.04495749]
Rank: 0, local id: 0, noise_sample 987: [-5.28918923  2.17772978]
Reached max iterations with fractional norm error 0.0038452121370390504.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 988: [6.34031129 0.42285442 0.25902199 0.04545509]
Rank: 0, local id: 0, noise_sample 988: [-5.38394303  2.10695843]
Reached max iterations with fractional norm error 0.0009510462912049172.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 989: [6.34040317 0.42313326 0.25947744 0.04677516]
Rank: 0, local id: 0, noise_sample 989: [-5.81745402  3.0261327 ]
Reached max iterations with fractional norm error 0.000503800804563707.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 990: [6.34088424 0.42306898 0.25902578 0.0470383

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]


Rank: 0, local id: 0, noise_sample 997: [-3.23187118  3.8319272 ]
Reached max iterations with fractional norm error 0.0056640924570494.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 998: [6.33989172 0.40985352 0.2835725  0.06050332]
Rank: 0, local id: 0, noise_sample 998: [-3.92003535  2.38764544]
Reached max iterations with fractional norm error 0.00017731099304487557.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 999: [6.33974124 0.42581593 0.27826069 0.04428273]
Rank: 0, local id: 0, noise_sample 999: [-3.5799036   3.13626581]
Reached max iterations with fractional norm error 0.00172967513555083.
